# Les bases du multithreading

## La programmation concurrente

- Trouver ce qui ne va pas dans ce programme :

In [14]:
fn main() {
    let mut data: i32 = 42;
    let _t = std::thread::spawn(move || {
        dbg!(data);
        data = 11;
        dbg!(data);
    });
    data = 22;
    dbg!(data);

    let mut s: String = "42".to_owned();
    let _t = std::thread::spawn(move || {
        dbg!(&s);
        s.push_str("11");
        dbg!(&s);
    });
    s.push_str("22");
    dbg!(&s);   
}

Error: borrow of moved value: `s`

Puisqu'une String n'est pas Copy, ma String s a été déplacée dans le thread qui a été crée. J'ai deux possibilités :

- Soit je renonce a utiliser la string s dans mon main() après avoir lancé le thread.
- Soit je fais un clone de s et j'envoie le clone à mon thread (ou le contraire).

## Le MPSC

- Dans cette partie, nous allons voir comment monter un programme multi thread en Rust. En plus du thread main, nous allons créer deux threads, un content et un pas content qui panic. Et plutôt que d'utiliser un bête mutex que vous avez déjà rencontré mille fois dans votre expérience de programmeur, nous allons utiliser un MPSC. Multiple Producer, Single Consumer. Nous ne chercherons pas non plus à joindre les threads, c'est trop banal aussi.

### Description

![MPSC](pictures/mpsc.png)

> Un MPSC est déjà thread safe, il est protégé en interne par un Mutex ou un sémaphore ou autre...Il faut regarder dans le code de la STD pour savoir ça ! Mais il est thread safe en tout cas.

> C'est un canal de communication, le pattern habituel consiste a donner un SENDER à chaque thread crées en plus du main() et d'utiliser le RECEIVER dans le thread principal. RECEIVER qui lui est unique. Single Receiver...

>```
>pub fn channel<T>() -> (Sender<T>, Receiver<T>)
>```

La fonction channel() retourne un Sender et un Receiver. Notez le generic T. Cela implique que l'on peut envoyer ou recevoir le type de donnée que l'on veut. Ici, nous allons renvoyer un tuple (enum + threadId) afin de passer un message ainsi que de connaître son producteur.

Il existe peut être déjà une méthode sur le Receiver pour voir le threadID...

### Initialisation

- Pour obtenir le Receiver et le Sender, on procède donc ainsi en précisant bien le type de donnée que l'on va transmettre. Notez que l'on clone déjà le sender car on sait qu'on va l'envoyer à deux threads. Autant cloner tant que le fer est chaud :

In [13]:
use std::sync::mpsc;
use std::thread::ThreadId;

#[derive(Debug, Copy, Clone)]
enum Event {
    Hello,
    HelloAgain,
    IWantToStop,
}

fn main() {
    let (sender, receiver): (
        mpsc::Sender<(Event, ThreadId)>,
        mpsc::Receiver<(Event, ThreadId)>,
    ) = mpsc::channel();
    let sender2 = sender.clone();
}

### Création des threads

- Aussi simple que ça: Notez le move et la closure || pour y penser plus tard. Un jour ces notions vous paraîtront clairs mais ne sont pas utiles pour l'instant :

In [12]:
let _happy_thread = std::thread::spawn(move || { 
});

let _bad_thread = std::thread::spawn(move || { 
});

- Voici des exemples d'utilisation du sender, dans le thread 1 et 2 respectifs :

**thread 1**
```
sender
    .send((Event::HelloAgain, thread::current().id()))
    .unwrap();
```

**thread 2**
```
sender2
    .send((Event::Hello, thread::current().id()))
    .unwrap();
```

- Et voici le code du receiver du thread principal :
```
loop {
    let (msg, id) = receiver.recv().unwrap();
    // if condition, break
}
```

> Il attend un message, l'appel est dit bloquant et fait un tour de loop jusqu'au next....

### Exécuter quelque chose dans son code

> L’idée peut être pour continuer et de faire communiquer régulièrement les threads, en attendant un peu entre chaque message grâce au sleep() :

![SLEEP](pictures/sleep.png)

> Peut être aussi quitter le programme quand un des threads nous dit qu'il en peut plus. (3e variant de l'enum...)

> Ou afficher le threadId pour savoir qui est vient de parler...


> Voir même faire panic! Un thread autre que le principal au bout d'un certain temps pour voir comment le programme se comporte !

In [11]:
panic!("sa mere");

thread '<unnamed>' panicked at 'sa mere', src/lib.rs:168:1
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: <unknown>
   3: <unknown>
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


**NB** Ce serait dommage qu'un thread se termine tout juste après sa création ...

### Code complet

In [10]:
use std::sync::mpsc;
use std::thread::ThreadId;
use std::{thread, time};

#[derive(Debug, Copy, Clone)]
enum Event {
    Hello,
    HelloAgain,
    IWantToStop,
}

fn main() {
    let (sender, receiver): (
        mpsc::Sender<(Event, ThreadId)>,
        mpsc::Receiver<(Event, ThreadId)>,
    ) = mpsc::channel();
    let sender2 = sender.clone();

    let three_seconds = time::Duration::from_millis(3000);

    let _happy_thread = thread::spawn(move || {
        sender.send((Event::Hello, thread::current().id())).unwrap();
        loop {
            thread::sleep(three_seconds);
            sender
                .send((Event::HelloAgain, thread::current().id()))
                .unwrap();
        }
    });

    let _bad_thread = thread::spawn(move || {
        sender2
            .send((Event::Hello, thread::current().id()))
            .unwrap();
        thread::sleep(three_seconds * 3);
        sender2
            .send((Event::IWantToStop, thread::current().id()))
            .unwrap();
        panic!("sa mere");
    });

    loop {
        let (msg, id) = receiver.recv().unwrap();
        println!("message recu: {:?} de {:?}", msg, id);
        if let Event::IWantToStop = msg {
            println!("A thread want to stop");
            break;
        }
    }
    println!("Exiting Main thread normallu. Bye");
}
main()

message recu: Hello de ThreadId(7)
message recu: Hello de ThreadId(8)
message recu: HelloAgain de ThreadId(7)
message recu: HelloAgain de ThreadId(7)


thread '<unnamed>' panicked at 'sa mere', src/lib.rs:46:9
stack backtrace:
thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: SendError { .. }', src/lib.rs:34:18


message recu: IWantToStop de ThreadId(8)
A thread want to stop
Exiting Main thread normallu. Bye


()

   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
stack backtrace:
   0: rust_begin_unwind
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/std/src/panicking.rs:575:5
   1: core::panicking::panic_fmt
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/panicking.rs:64:14
   2: core::result::unwrap_failed
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/result.rs:1790:5
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


> Le panic est recherché dans le code !

**N'oubliez pas que Rust est extrêmement puissant dans la gestion de ses threads, ce n'est pas pour rien que l'on parle de programmation concurrente.  Tant qu'il n'y a pas de code explicitement unsafe, les threads tiennent bon. Les règles strictes de Rust qui nous font maudire ce langage lorsque nous sommes en monothreads sont de vraies bénédictions en environnement multithreaded.**

## Le trait Send

- Rust détermine les types qui peuvent être passés a un thread **en fonction de s'ils implémentent le trait Send**. Aussi, si une structure possède plusieurs éléments qui implémentent tous le trait Send, le compilateur considéra que la structure est Send, on appelle cela un **AUTO trait** :

In [9]:
// Tous les types contenus ici sont Send
#[derive(Debug)]
struct S {
    i: u8,
    j: u8,
    k: String,
}
let s = S {
    i: 1,
    j: 2,
    k: "toto".to_string(),
};
let _t = std::thread::spawn(move || {
    dbg!(s);
});

[src/lib.rs:123] s = S {
    i: 1,
    j: 2,
    k: "toto",
}


- Il y a quelques types qui ne sont pas Send comme les raw pointers, ou les RC ou simple Reference Counter (a ne pas confondre avec les ARC). Si l'on veut vraiment passer ces types à un thread, l'on sera obligé d'implémenter Send manuellement pour une structure englobant ces types, cette instruction est par nature UNSAFE :

In [8]:
// Un pointeur en rust n'est pas Send
#[derive(Debug)]
struct S1 {
    ptr: *const u8,
}
let s1 = S1 {
    ptr: std::ptr::null(),
};
let _t = std::thread::spawn(move || {
    dbg!(s1);
});

Error: `*const u8` cannot be sent between threads safely

> Après ajout de l'implémentation unsafe du trait Send pour la struct s2 :

In [7]:
// Un pointeur en rust n'est pas Send
#[derive(Debug)]
struct S2 {
    ptr: *const u8,
}

unsafe impl Send for S2 {} // AJOUT POUR QUE CA COMPILE

let s2 = S2 {
    ptr: std::ptr::null(),
};
let _t = std::thread::spawn(move || {
    dbg!(s2);
});

[src/lib.rs:115] s2 = S2 {
    ptr: 0x0000000000000000,
}


> Comme tout ce qui est `unsafe`, le codeur doit en assumer les risques ! Il peut y avoir de bonnes raisons ici qu'un pointeur ne soit pas Send !